In [1]:
import numpy as np

def ricker( duration, dt, f ):
    freq = np.array( f )
    t = np.arange( -duration/2, duration/2 , dt)
    output = np.zeros( (t.size, freq.size ) )
    for i in range(freq.size):
        pi2 = (np.pi ** 2.0)
        if ( freq.size == 1 ):
            fsqr = freq ** 2.0
        else:
            fsqr = freq[i] ** 2.0
        tsqr = t ** 2.0
        pft = pi2 * fsqr * tsqr
        A = (1 - (2 * pft)) * np.exp(-pft)
        output[:,i] = A
    if freq.size == 1: output = output.flatten()
    return output

In [6]:
%matplotlib inline
import sys, os
import matplotlib.pyplot as plt

path_to_files = 'data_fold_dyke_fault/su/'
files = os.listdir(path_to_files)
su_files = [x for x in files if x.find(".su") != -1] 

yz_files = [x for x in su_files if x.find("yz") != -1]
yz_size = (400, 200)

xz_files = [x for x in su_files if x.find("xz") != -1]
xz_size = (280, 200)

w = ricker(0.25, 0.004, 10)

def convolve(w, file, slice_shape):
    print("processing", file)
    slice = np.fromfile(path_to_files + file, dtype=np.float64)
    data = np.transpose(slice.reshape(slice_shape))
    reflectivity = np.apply_along_axis(lambda x: np.convolve(x, [-0.5, 0.5], mode='same'), 0, data)
    seismic = np.apply_along_axis(lambda x: np.convolve(x, w, mode='same'), 0, reflectivity)
    return seismic

seismic_path = path_to_files+'seismic/'
model_path = path_to_files+'model/'

if not os.path.exists(seismic_path):
    os.makedirs(seismic_path)
    
if not os.path.exists(model_path):
    os.makedirs(model_path)

for f in xz_files:
    seismic = convolve(w, f, xz_size)
    np.save(model_path+f, f)
    np.save(seismic_path+f, seismic)
    
for f in yz_files:
    seismic = convolve(w, f, yz_size)
    np.save(model_path+f, f)
    np.save(seismic_path+f, seismic)
    
plt.figure(figsize=(16,16))
plt.subplot(2,2,1)
plt.plot(w)
plt.subplot(2,2,2)
plt.imshow(data)
plt.subplot(2,2,4)
plt.imshow(seismic)

processing out_0001_xz0.su


ValueError: cannot reshape array of size 56000 into shape (496,200)